## Visualização e Correção de Dados

In [5]:
import pandas as pd 
import numpy as np

In [6]:
# --- Carregar o Dataset ---
try:
    df = pd.read_csv('Ibovespa_Consolidado.csv')
    print("Arquivo 'Ibovespa_Consolidado.csv' carregado com sucesso!")
except FileNotFoundError:
    print("Erro: O arquivo 'Ibovespa_Consolidado.csv' não foi encontrado.")
    print("Verifique se o nome do arquivo está correto e se ele está na mesma pasta que este script.")
    exit() # Encerra o script se o arquivo não for encontrado



Arquivo 'Ibovespa_Consolidado.csv' carregado com sucesso!


In [7]:
# --- . Visualização Inicial dos Dados ---
print("\n--- Visualização Inicial do Dataset ---")
print("\nPrimeiras 5 linhas do dataset:")
print(df.head())

print("\nInformações gerais do dataset (tipos de dados, valores nulos):")
df.info()

print("\nContagem de valores nulos por coluna:")
print(df.isnull().sum())


--- Visualização Inicial do Dataset ---

Primeiras 5 linhas do dataset:
         Data  Último  Abertura  Máxima  Mínima     Vol.    Var%
0  01.08.2000  16.290    16.475  16.591  16.127  123,64M  -1,00%
1  31.07.2000  16.455    16.487  16.713  16.455  705,80M  -0,19%
2  28.07.2000  16.486    16.949  17.039  16.448  102,09M  -2,73%
3  27.07.2000  16.949    17.121  17.225  16.923  110,50M  -1,00%
4  26.07.2000  17.121    17.049  17.211  16.840  123,20M   0,42%

Informações gerais do dataset (tipos de dados, valores nulos):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10756 entries, 0 to 10755
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Data      10756 non-null  object 
 1   Último    10756 non-null  float64
 2   Abertura  10756 non-null  float64
 3   Máxima    10756 non-null  float64
 4   Mínima    10756 non-null  float64
 5   Vol.      10755 non-null  object 
 6   Var%      10756 non-null  object 
dtypes: float6

In [10]:
# --- . Limpeza e Conversão de Tipos ---
print("\n--- Iniciando Limpeza e Conversão de Tipos ---")

# a) Converter a coluna 'Data' para o formato de data (datetime)
# O formato atual é dia.mês.ano ('%d.%m.%Y')
df['Data'] = pd.to_datetime(df['Data'], format='%d.%m.%Y')
print("\nColuna 'Data' convertida para o formato datetime.")

# b) Limpar e converter colunas numéricas ('Último', 'Abertura', 'Máxima', 'Mínima')
# Essas colunas estão como strings (object) e usam '.' como separador de milhar.
colunas_numericas = ['Último', 'Abertura', 'Máxima', 'Mínima']
print("\nConvertendo colunas para formato numérico...")

for coluna in colunas_numericas:
    # VERIFICAÇÃO: O dtype 'object' é como o pandas representa colunas de texto.
    if df[coluna].dtype == 'object':
        # Se for texto, removemos o '.' e convertemos para float.
        print(f"Limpando e convertendo a coluna '{coluna}' (era texto)...")
        df[coluna] = df[coluna].str.replace('.', '', regex=False).astype(float)
    else:
        # Se JÁ for um número, apenas garantimos que está como float.
        print(f"Coluna '{coluna}' já está em formato numérico. Apenas garantindo que seja float.")
        df[coluna] = df[coluna].astype(float)

print(f"Colunas {colunas_numericas} foram processadas com sucesso.")

# c) Limpar e converter a coluna 'Var%'
# Está como string, com ',' para decimal e '%' no final.
df['Var%'] = df['Var%'].str.replace(',', '.', regex=False).str.replace('%', '', regex=False).astype(float) / 100
print("Coluna 'Var%' convertida para formato numérico (float) e dividida por 100.")

# d) Limpar e converter a coluna 'Vol.' (Volume)
# É uma string com 'B' para bilhões e 'M' para milhões, e ',' como decimal.
def converter_volume(volume_str):
    if isinstance(volume_str, str):
        volume_str = volume_str.strip().upper()
        volume_str = volume_str.replace(',', '.')
        if 'B' in volume_str:
            return float(volume_str.replace('B', '')) * 1_000_000_000
        elif 'M' in volume_str:
            return float(volume_str.replace('M', '')) * 1_000_000
        elif 'K' in volume_str:
             return float(volume_str.replace('K', '')) * 1_000
    return volume_str

df['Vol.'] = df['Vol.'].apply(converter_volume)
# Converte a coluna para numérico, e os valores que não puderam ser convertidos viram NaN (Not a Number)
df['Vol.'] = pd.to_numeric(df['Vol.'], errors='coerce')
print("Coluna 'Vol.' convertida para formato numérico.")


--- Iniciando Limpeza e Conversão de Tipos ---

Coluna 'Data' convertida para o formato datetime.

Convertendo colunas para formato numérico...
Coluna 'Último' já está em formato numérico. Apenas garantindo que seja float.
Coluna 'Abertura' já está em formato numérico. Apenas garantindo que seja float.
Coluna 'Máxima' já está em formato numérico. Apenas garantindo que seja float.
Coluna 'Mínima' já está em formato numérico. Apenas garantindo que seja float.
Colunas ['Último', 'Abertura', 'Máxima', 'Mínima'] foram processadas com sucesso.
Coluna 'Var%' convertida para formato numérico (float) e dividida por 100.
Coluna 'Vol.' convertida para formato numérico.


In [ ]:
# --- . Tratamento de Valores Nulos (NaN) ---
print("\n--- Tratando Valores Nulos (NaN) ---")
print(f"\nValores nulos antes do tratamento:\n{df.isnull().sum()}")

# Para dados de séries temporais, uma boa estratégia é preencher os valores ausentes
# com o último valor válido conhecido (forward fill).
# Isso é especialmente útil para preços de ações (Abertura, Máxima, etc.)
colunas_para_preencher = ['Último', 'Abertura', 'Máxima', 'Mínima', 'Vol.', 'Var%']
df[colunas_para_preencher] = df[colunas_para_preencher].fillna(method='ffill')

# Se ainda sobrar algum NaN no início do dataset, podemos usar o backfill
df[colunas_para_preencher] = df[colunas_para_preencher].fillna(method='bfill')

print(f"\nValores nulos depois do tratamento:\n{df.isnull().sum()}")


--- Tratando Valores Nulos (NaN) ---

Valores nulos antes do tratamento:
Data        0
Último      0
Abertura    0
Máxima      0
Mínima      0
Vol.        1
Var%        0
dtype: int64

Valores nulos depois do tratamento:
Data        0
Último      0
Abertura    0
Máxima      0
Mínima      0
Vol.        0
Var%        0
dtype: int64


C:\Users\zirn\AppData\Local\Temp\ipykernel_38980\4108165767.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[colunas_para_preencher] = df[colunas_para_preencher].fillna(method='ffill')
C:\Users\zirn\AppData\Local\Temp\ipykernel_38980\4108165767.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[colunas_para_preencher] = df[colunas_para_preencher].fillna(method='bfill')


In [14]:
# ---  Verificar e Tratar Datas Duplicadas ---
print("\n--- Verificando se existem datas duplicadas ---")

# A propriedade `index.duplicated()` cria uma série booleana (True/False).
# `keep=False` marca TODAS as ocorrências de uma duplicata como True.
duplicatas = df.index.duplicated(keep=False)

# .any() verifica se existe algum valor True na série de duplicatas
if duplicatas.any():
    print(f"\nATENÇÃO: Foram encontradas datas duplicadas!")
    # df[duplicatas] filtra o dataframe para mostrar apenas as linhas onde o índice é duplicado.
    print("Abaixo estão as linhas com datas repetidas:")
    print(df[duplicatas])

    # --- Tratamento das duplicatas ---
    # A abordagem mais comum para dados financeiros é manter o último registro do dia.
    print("\nRemovendo as datas duplicadas e mantendo o último registro de cada data...")
    df = df[~df.index.duplicated(keep='last')]
    
    print("Duplicatas removidas com sucesso.")
    
else:
    print("Ótima notícia: Nenhuma data duplicada foi encontrada.")


--- Verificando se existem datas duplicadas ---

ATENÇÃO: Foram encontradas datas duplicadas!
Abaixo estão as linhas com datas repetidas:
            Último  Abertura  Máxima  Mínima         Vol.  Var%
Data                                                           
2000-08-01   16.29    16.475  16.591  16.127  123640000.0 -0.01
2000-08-01   16.29    16.475  16.591  16.127  123640000.0 -0.01

Removendo as datas duplicadas e mantendo o último registro de cada data...
Duplicatas removidas com sucesso.


In [15]:
# --- . Organização Final do Dataset ---
print("\n--- Organizando o Dataset ---")

# a) Definir a coluna 'Data' como o índice do dataframe
df = df.set_index('Data')
print("\nColuna 'Data' definida como o novo índice do dataframe.")

# b) Ordenar o dataframe pela data (do mais antigo para o mais novo)
df = df.sort_index(ascending=True)
print("Dataset ordenado pela data em ordem crescente.")



--- Organizando o Dataset ---


KeyError: "None of ['Data'] are in the columns"

In [13]:
# --- 6. Visualização do Dataset Limpo ---
print("\n--- Visualização Final do Dataset Limpo ---")
print("\nPrimeiras 5 linhas do dataset limpo e organizado:")
print(df.head())

print("\nÚltimas 5 linhas do dataset limpo e organizado:")
print(df.tail())

print("\nInformações finais do dataset:")
df.info()


--- Visualização Final do Dataset Limpo ---

Primeiras 5 linhas do dataset limpo e organizado:
            Último  Abertura  Máxima  Mínima         Vol.  Var%
Data                                                           
1982-01-07     0.0       0.0     0.0     0.0  116090000.0   0.0
1982-01-08     0.0       0.0     0.0     0.0  329760000.0   0.0
1982-01-11     0.0       0.0     0.0     0.0  127520000.0   0.0
1982-01-12     0.0       0.0     0.0     0.0  122620000.0   0.0
1982-01-13     0.0       0.0     0.0     0.0  106190000.0   0.0

Últimas 5 linhas do dataset limpo e organizado:
             Último  Abertura   Máxima   Mínima          Vol.    Var%
Data                                                                 
2025-07-28  132.129   133.538  133.902  131.550  6.630000e+09 -0.0104
2025-07-29  132.726   132.130  133.346  132.130  6.320000e+09  0.0045
2025-07-30  133.990   132.702  134.368  131.883  8.660000e+09  0.0095
2025-07-31  133.071   133.987  133.987  132.096  9.200000

In [16]:
# --- 7. Salvar o Dataset Limpo ---
nome_arquivo_limpo = 'Ibovespa.csv'
df.to_csv(nome_arquivo_limpo)
print(f"\n--- Processo Finalizado! ---")
print(f"O dataset limpo e organizado foi salvo como '{nome_arquivo_limpo}'.")


--- Processo Finalizado! ---
O dataset limpo e organizado foi salvo como 'Ibovespa.csv'.
